## Notebook to calculate all important metrics for the networks and save them

In [54]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa

# imported own functions
from utils import market_values, capacity, capacity_links, capacity_storage_units, generation, generation_links, \
    generation_storage_units, market_values_storage_units, market_values_links, time_stored_LIFO

# imported own definitions
from utils import resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal

# general variables
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Regions
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]

Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [55]:
# calc market values, generation, lmps, capacity factors for generators, links and storage units
for n in [stst, exp]:
    df_regions_onshore = onshore_regions.copy()
    df_regions_offshore = offshore_regions.copy()

    # function for carriers in n.generators.carrier.unique() # 13 / 13
    for carrier in n.generators.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_offshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        df_regions_offshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        # lmps
        # capacity factors This calculation is correct? as capacity is multiplied by 2920 is the same as multiplying the generation by 3 and then dividing it by the capacity times 8760 (as cap is in MWh?)
        df_regions_onshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)
        df_regions_offshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)

    # function for carriers in n.links.carrier.unique() # 53 / 55
    for carrier in n.links.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_links(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_links(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cf"] = generation_links(n, carrier) / (capacity_links(n, carrier) * 2920)

    # function for carriers in n.storage_units.carrier.unique() # 2 / 2
    for carrier in n.storage_units.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_storage_units(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_storage_units(n, carrier) / 1000 * 3
        # capacity factors (both generation and consumption(loading) is considered
        gen = abs(n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier])
        gen.columns = gen.columns.map(n.storage_units.bus)
        gen.columns = gen.columns.map(n.buses.location)
        df_regions_onshore[f"{carrier}_cf"] = gen.sum() / (capacity_storage_units(n, carrier) * 2920)

    # set market values to nan where generation in corresponding region is lower than % quantile
    qt = 0.2
    for carrier in (n.generators.carrier.unique().tolist() +
                    n.links.carrier.unique().tolist() +
                    n.storage_units.carrier.unique().tolist()):
        index = df_regions_onshore[f"{carrier}_gen"] <= np.nanquantile(df_regions_onshore[f"{carrier}_gen"], qt)
        df_regions_onshore[f"{carrier}_mv"][index] = np.nan

    # calc lmps at the buses (lmps that are only present for EU (e.g. oil) are nan at the moment)
    # TODO: decide if EU lmps are used as lmp for all regions
    for carrier_bus in n.buses.carrier.unique():
        # index would be same names as the bus (not the location as it is in the index of
        # df_regions_onshore so far -> map location to make sure the right lmp is set
        locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
        lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index].mean()
        df = pd.concat([lmps, locs], axis=1).rename(columns={0: f"{carrier_bus}_lmp"})
        df.set_index("location", inplace=True)
        if df.size == 1:
            if df.index == "EU":
                df = pd.DataFrame(np.repeat(df.values, 181), index=df_regions_onshore.index,
                                  columns=[f"{carrier_bus}_lmp"])
                df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]
        else:
            df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]

    if n == stst:
        df_stst_ons = df_regions_onshore
        df_stst_off = df_regions_offshore

    if n == exp:
        df_exp_ons = df_regions_onshore
        df_exp_off = df_regions_offshore

C:\Users\Julian\Programs\anaconda3\envs\ma_env\lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Julian\Programs\anaconda3\envs\ma_env\lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

C:\Users\Julian\Programs\anaconda3\envs\ma_env\lib\site-packages\geopandas\geodataframe.py:1443: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [56]:
df_stst_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,urban central water tanks_lmp,biogas_lmp,solid biomass_lmp,solid biomass for industry_lmp,gas for industry_lmp,H2 liquid_lmp,oil_lmp,process emissions_lmp,low voltage_lmp,home battery_lmp
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,NaN,0.029238,0.090705,NaN,0.015125,0.038018,44.628736,...,43.532557,113.716896,102.453861,78.924392,113.135704,137.617875,50.009981,-408.520212,100.338046,100.380697
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,83.457109,30395.904252,0.149667,54.679083,...,55.231280,113.716896,102.453861,78.924392,113.135704,151.448316,50.009981,-408.520212,115.006246,115.044440
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,0.000348,0.004685,65.745903,...,55.553668,113.716896,102.453861,78.924392,113.135704,154.026970,50.009981,-408.520212,121.910720,121.947576
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,0.014320,0.039178,58.223787,...,58.936874,113.716896,102.453861,78.924392,113.135704,148.663198,50.009981,-408.520212,112.352878,112.386673
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,0.014059,0.031802,51.160537,...,51.074197,113.716896,102.453861,78.924392,113.135704,147.689672,50.009981,-408.520212,112.040690,112.006725


In [57]:
df_stst_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,services urban decentral solar thermal_cf,urban central solar thermal_mv,urban central solar thermal_gen,urban central solar thermal_cf,oil_mv,oil_gen,oil_cf,solar rooftop_mv,solar rooftop_gen,solar rooftop_cf
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,126.148417,0.029238,0.090705,131.075254,0.015125,0.038018,44.628736,...,0.049208,45.096866,73.985333,0.038386,NaN,NaN,NaN,56.499070,2987.307119,0.129410
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,78.122119,6139.041623,0.393274,73.103671,9140.702915,0.312504,58.613241,...,0.025145,34.889495,0.044487,0.027616,NaN,NaN,NaN,69.372367,2060.164490,0.095390
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,95.498412,0.148386,0.176485,87.582734,11052.570807,0.142613,45.281165,...,0.046705,31.907542,7.046495,0.053815,NaN,NaN,NaN,52.951120,3345.966155,0.124563
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,53.873868,0.669227,0.364074,44.498808,43594.793974,0.295848,62.960328,...,0.025180,17.588450,0.014318,0.018676,NaN,NaN,NaN,70.716513,0.226622,0.078537
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,62.126430,0.790338,0.319996,52.030347,89617.262854,0.320665,62.003765,...,0.024803,29.140615,0.020005,0.020499,NaN,NaN,NaN,71.601688,0.880078,0.084488


In [58]:
df_exp_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,urban central water tanks_lmp,biogas_lmp,solid biomass_lmp,solid biomass for industry_lmp,gas for industry_lmp,H2 liquid_lmp,oil_lmp,process emissions_lmp,low voltage_lmp,home battery_lmp
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,NaN,0.016331,0.087899,NaN,0.016423,0.043458,43.422954,...,40.254427,98.36817,67.610155,53.497697,104.938187,116.041295,50.010024,-371.637315,85.769520,85.775386
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,79.253841,0.567355,0.144707,55.129859,...,46.058782,98.36817,67.610155,53.497697,104.938187,117.751542,50.010024,-371.637315,90.810056,90.843749
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,0.000326,0.004923,57.245535,...,46.935243,98.36817,67.610155,53.497697,104.938187,117.270318,50.010024,-371.637315,89.167959,89.169977
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,0.012936,0.044168,54.745027,...,46.913558,98.36817,67.610155,53.497697,104.938187,117.470578,50.010024,-371.637315,89.668622,89.663995
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,0.010620,0.031748,51.151356,...,43.644096,98.36817,67.610155,53.497697,104.938187,117.017758,50.010024,-371.637315,89.403708,89.423589


In [59]:
df_exp_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,services urban decentral solar thermal_cf,urban central solar thermal_mv,urban central solar thermal_gen,urban central solar thermal_cf,oil_mv,oil_gen,oil_cf,solar rooftop_mv,solar rooftop_gen,solar rooftop_cf
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,106.977597,0.016331,0.087899,106.502516,0.016423,0.043458,43.422954,...,0.052464,27.814360,128.538879,0.062264,NaN,NaN,NaN,55.050254,3018.547275,0.130763
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,56.352353,6343.446483,0.406369,52.212890,9585.550789,0.327713,58.383401,...,0.025774,29.938956,0.020416,0.027160,NaN,NaN,NaN,66.894575,1.726123,0.092860
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,94.568790,0.100601,0.178239,85.769797,8.556417,0.144543,44.802486,...,0.048685,28.496283,1.715255,0.059337,NaN,NaN,NaN,51.305046,3363.626125,0.125219
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,51.455031,8441.804726,0.434748,47.257302,45018.091917,0.305507,60.991588,...,0.024480,33.265504,0.021125,0.026457,NaN,NaN,NaN,66.137206,0.083125,0.076325
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,60.568652,0.306136,0.308317,50.245283,89980.590188,0.321965,61.958604,...,0.024536,31.269660,0.019227,0.025714,NaN,NaN,NaN,66.952632,0.143762,0.080772


In [60]:
# save to pickle
df_stst_ons.to_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off.to_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons.to_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off.to_pickle("../data/processed/df_exp_off.pkl")

In [61]:
# verify reload
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

In [62]:
df_stst_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,urban central water tanks_lmp,biogas_lmp,solid biomass_lmp,solid biomass for industry_lmp,gas for industry_lmp,H2 liquid_lmp,oil_lmp,process emissions_lmp,low voltage_lmp,home battery_lmp
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,NaN,0.029238,0.090705,NaN,0.015125,0.038018,44.628736,...,43.532557,113.716896,102.453861,78.924392,113.135704,137.617875,50.009981,-408.520212,100.338046,100.380697
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,83.457109,30395.904252,0.149667,54.679083,...,55.231280,113.716896,102.453861,78.924392,113.135704,151.448316,50.009981,-408.520212,115.006246,115.044440
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,0.000348,0.004685,65.745903,...,55.553668,113.716896,102.453861,78.924392,113.135704,154.026970,50.009981,-408.520212,121.910720,121.947576
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,0.014320,0.039178,58.223787,...,58.936874,113.716896,102.453861,78.924392,113.135704,148.663198,50.009981,-408.520212,112.352878,112.386673
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,0.014059,0.031802,51.160537,...,51.074197,113.716896,102.453861,78.924392,113.135704,147.689672,50.009981,-408.520212,112.040690,112.006725


In [63]:
df_stst_ons.head().dtypes

geometry                 geometry
coords                     object
name                       object
offwind-ac_mv             float64
offwind-ac_gen            float64
                           ...   
H2 liquid_lmp             float64
oil_lmp                   float64
process emissions_lmp     float64
low voltage_lmp           float64
home battery_lmp          float64
Length: 233, dtype: object

In [64]:
df_stst_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,services urban decentral solar thermal_cf,urban central solar thermal_mv,urban central solar thermal_gen,urban central solar thermal_cf,oil_mv,oil_gen,oil_cf,solar rooftop_mv,solar rooftop_gen,solar rooftop_cf
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,126.148417,0.029238,0.090705,131.075254,0.015125,0.038018,44.628736,...,0.049208,45.096866,73.985333,0.038386,NaN,NaN,NaN,56.499070,2987.307119,0.129410
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,78.122119,6139.041623,0.393274,73.103671,9140.702915,0.312504,58.613241,...,0.025145,34.889495,0.044487,0.027616,NaN,NaN,NaN,69.372367,2060.164490,0.095390
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,95.498412,0.148386,0.176485,87.582734,11052.570807,0.142613,45.281165,...,0.046705,31.907542,7.046495,0.053815,NaN,NaN,NaN,52.951120,3345.966155,0.124563
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,53.873868,0.669227,0.364074,44.498808,43594.793974,0.295848,62.960328,...,0.025180,17.588450,0.014318,0.018676,NaN,NaN,NaN,70.716513,0.226622,0.078537
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,62.126430,0.790338,0.319996,52.030347,89617.262854,0.320665,62.003765,...,0.024803,29.140615,0.020005,0.020499,NaN,NaN,NaN,71.601688,0.880078,0.084488


In [65]:
df_exp_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,urban central water tanks_lmp,biogas_lmp,solid biomass_lmp,solid biomass for industry_lmp,gas for industry_lmp,H2 liquid_lmp,oil_lmp,process emissions_lmp,low voltage_lmp,home battery_lmp
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,NaN,0.016331,0.087899,NaN,0.016423,0.043458,43.422954,...,40.254427,98.36817,67.610155,53.497697,104.938187,116.041295,50.010024,-371.637315,85.769520,85.775386
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,79.253841,0.567355,0.144707,55.129859,...,46.058782,98.36817,67.610155,53.497697,104.938187,117.751542,50.010024,-371.637315,90.810056,90.843749
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,0.000326,0.004923,57.245535,...,46.935243,98.36817,67.610155,53.497697,104.938187,117.270318,50.010024,-371.637315,89.167959,89.169977
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,0.012936,0.044168,54.745027,...,46.913558,98.36817,67.610155,53.497697,104.938187,117.470578,50.010024,-371.637315,89.668622,89.663995
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,0.010620,0.031748,51.151356,...,43.644096,98.36817,67.610155,53.497697,104.938187,117.017758,50.010024,-371.637315,89.403708,89.423589


In [66]:
df_exp_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cf,solar_mv,...,services urban decentral solar thermal_cf,urban central solar thermal_mv,urban central solar thermal_gen,urban central solar thermal_cf,oil_mv,oil_gen,oil_cf,solar rooftop_mv,solar rooftop_gen,solar rooftop_cf
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,106.977597,0.016331,0.087899,106.502516,0.016423,0.043458,43.422954,...,0.052464,27.814360,128.538879,0.062264,NaN,NaN,NaN,55.050254,3018.547275,0.130763
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,56.352353,6343.446483,0.406369,52.212890,9585.550789,0.327713,58.383401,...,0.025774,29.938956,0.020416,0.027160,NaN,NaN,NaN,66.894575,1.726123,0.092860
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,94.568790,0.100601,0.178239,85.769797,8.556417,0.144543,44.802486,...,0.048685,28.496283,1.715255,0.059337,NaN,NaN,NaN,51.305046,3363.626125,0.125219
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,51.455031,8441.804726,0.434748,47.257302,45018.091917,0.305507,60.991588,...,0.024480,33.265504,0.021125,0.026457,NaN,NaN,NaN,66.137206,0.083125,0.076325
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,60.568652,0.306136,0.308317,50.245283,89980.590188,0.321965,61.958604,...,0.024536,31.269660,0.019227,0.025714,NaN,NaN,NaN,66.952632,0.143762,0.080772
